## DB를 Chroma 에서 Pinecone 으로 변경
- Chroma 는 로컬에 저장
- Pinecone 는 클라우드 방식
- 코드는 2.1, 2.2 와 거의 비슷하며, db에 대한 코드만 다름

In [1]:
# Pinecone 필요한 패키지 설치

!pip install --upgrade --quiet  \
    langchain-pinecone \
    langchain-openai \
    langchain \
    langchain-community \
    pinecone-notebooks


In [17]:
# 환경변수 설정

from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

True

In [18]:
# 문서를 쪼개어 chunk 로 나눔
# ecursively split by character 이용

# docx 로더를 임포트, Recursively split by character 를 임포트
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 스플리터 객체 생성
# 청크 사이즈 : 한 청크에 들어갈 토큰 수
# 청크 오버랩 : 완전히 분리되도록 청크를 나누는 것이 아니라 겹치도록 가져와서 가능성을 높임
recursive_text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

# tax.docx 를 읽어들인다.
loader=Docx2txtLoader("tax.docx")

# 텍스트 스플리터를 이용해 docx 문서를 읽고 청크로 나눈다.
document_list=loader.load_and_split(text_splitter=recursive_text_splitter)

In [19]:
document_list[:5], len(document_list)

([Document(page_content='소득세법\n\n소득세법\n\n[시행 2024. 7. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217\n\n\n\n제1장 총칙 <개정 2009. 12. 31.>\n\n\n\n제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n\n[본조신설 2009. 12. 31.]\n\n[종전 제1조는 제2조로 이동 <2009. 12. 31.>]\n\n\n\n제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>\n\n1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.\n\n2. “비거주자”란 거주자가 아닌 개인을 말한다.\n\n3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.\n\n4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.\n\n5. “사업자”란 사업소득이 있는 거주자를 말한다.\n\n② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.\n\n[본조신설 2009. 12. 31.]\n\n\n\n제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.\n\n1. 거주자\n\n2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인\n\n② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다

In [20]:
# upstage 임베딩을 이용해 임베딩 진행

# upstage 임베딩을 가져옴
from langchain_upstage import UpstageEmbeddings

# https://platform.openai.com/docs/guides/embeddings
# 위 사이트에서 최신 모델을 사용 --> text-embedding-3-large (이는 openai)


# 아래는 upstage
# 모델은 "solar-embedding-1-large"

embedding=UpstageEmbeddings(model="solar-embedding-1-large")

embedding

UpstageEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001FD1015E810>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001FD10133410>, model='solar-embedding-1-large', dimensions=None, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar', embedding_ctx_length=4096, embed_batch_size=10, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, http_client=None, http_async_client=None)

# 이 다음에 데이터 베이스 부분

In [21]:
# 그 다음이 데이터 베이스임
# 여기가 달라지는 부분임
# pinecone 에서 인덱스 까지 생성하고 난 후 진행


# 아래는 pinecone객체 생성
# 이 부분은 동작에는 필요 없는 부분이나,
# 공식문서에 있어서 실행해봄

import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# 생성한 인덱스
index_name="tax-index"

pine_api_key=os.environ.get("PINECONE_API_KEY")
pc=Pinecone(api_key=pine_api_key)

pc

In [22]:
# pinecone 데이터베이스 생성
# 클라우드에 올려짐

database = PineconeVectorStore.from_documents(
    document_list, 
    embedding, 
    index_name=index_name
)

In [23]:
# openai 키가 아니라, upstage 키를 사용함

# 라이브러리 로딩
# 이 부분에서 어떤 것을 가져와도 가능함
# 즉, 연동이 쉬움 (openai, upstage 둘다 가능함)
from langchain_upstage import ChatUpstage

# upstage의 api를 이용해 llm 모델 불러옴
llm=ChatUpstage(api_key="up_D8sLGDa0x7FknzDfrMuiVMxEObmEP")

llm

ChatUpstage(client=<openai.resources.chat.completions.Completions object at 0x000001FD100ECE90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FD100AA650>, upstage_api_key=SecretStr('**********'))

In [24]:
# 프롬프트 로딩
from langchain import hub

prompt=hub.pull("rlm/rag-prompt")

In [25]:
# QA Chain 생성

from langchain.chains import RetrievalQA

qa_chain=RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}

)

In [26]:
# 질문
query="연봉 5천만원인 직장인의 소득세는 얼마인가요?"

In [27]:
# # 질문하기

# message=qa_chain({"query": query})

# message

c:\최종 프로젝트\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉이 5천만원인 직장인의 소득세는 1,092,000원입니다.'}

: 